<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/CNN_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [4]:
model = ConvNeuralNet(num_classes)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)

In [12]:
total_step

782

In [5]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

            
            


Epoch [1/20], Loss: 1.3703
Epoch [2/20], Loss: 1.5997
Epoch [3/20], Loss: 1.6291
Epoch [4/20], Loss: 1.3691
Epoch [5/20], Loss: 0.9931
Epoch [6/20], Loss: 0.6872
Epoch [7/20], Loss: 0.9764
Epoch [8/20], Loss: 0.9943
Epoch [9/20], Loss: 0.9651
Epoch [10/20], Loss: 1.1746
Epoch [11/20], Loss: 0.6633
Epoch [12/20], Loss: 0.7530
Epoch [13/20], Loss: 0.7830
Epoch [14/20], Loss: 0.5743
Epoch [15/20], Loss: 0.8104
Epoch [16/20], Loss: 0.5616
Epoch [17/20], Loss: 0.6485
Epoch [18/20], Loss: 0.5207
Epoch [19/20], Loss: 0.5151
Epoch [20/20], Loss: 1.0170


In [6]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))




Accuracy of the network on the 50000 train images: 83.886 %


In [7]:
device

device(type='cpu')

In [8]:
images

tensor([[[[ 1.5642e+00,  1.5061e+00,  8.2760e-01,  ...,  1.5836e+00,
            1.6418e+00,  1.6805e+00],
          [ 1.4867e+00,  1.4285e+00,  2.4605e-01,  ...,  1.5836e+00,
            1.6418e+00,  1.6612e+00],
          [ 1.8162e+00,  1.8356e+00,  9.4391e-01,  ...,  1.7581e+00,
            1.6805e+00,  1.6418e+00],
          ...,
          [ 1.7581e+00,  1.7775e+00,  1.8744e+00,  ...,  1.8938e+00,
            1.9325e+00,  1.9132e+00],
          [ 1.7968e+00,  1.7775e+00,  1.9325e+00,  ...,  1.5642e+00,
            1.6418e+00,  1.4867e+00],
          [ 1.6418e+00,  1.7775e+00,  1.8356e+00,  ...,  1.3316e+00,
            1.3704e+00,  1.1378e+00]],

         [[ 1.4364e+00,  1.3381e+00,  5.5144e-01,  ...,  1.5348e+00,
            1.5741e+00,  1.6134e+00],
          [ 1.3971e+00,  1.2988e+00,  7.6703e-04,  ...,  1.4954e+00,
            1.5741e+00,  1.5938e+00],
          [ 1.5938e+00,  1.6331e+00,  7.2844e-01,  ...,  1.5938e+00,
            1.5544e+00,  1.5348e+00],
          ...,
     

In [9]:
labels

tensor([4, 0, 5, 4, 2, 1, 0, 7, 7, 0, 8, 3, 1, 3, 9, 5])

In [10]:
outputs

tensor([[-0.9070, -7.2098,  1.2129,  3.5827,  3.9381,  3.3199,  2.4668,  2.2104,
         -6.2647, -0.5280],
        [ 8.0060,  0.7814,  1.2714, -1.3597, -0.2095, -2.5268, -5.6638, -3.5197,
         -1.7399,  4.4888],
        [-5.2149, -0.3480,  0.0865,  2.4741,  2.1745,  3.4927,  1.2277, -0.9231,
         -1.1453,  0.3829],
        [-0.2915, -1.4756, -0.2789, -0.6190,  5.5814,  1.1299, -3.9164,  2.1680,
         -0.4774, -0.4979],
        [-0.8805, -4.7927,  4.9196,  1.2989,  2.6460,  1.8795,  3.3143, -1.2460,
         -3.6165, -2.6464],
        [-0.2506,  6.0306, -0.0350,  1.0120, -0.5585,  0.9661,  3.1212, -4.2047,
         -4.0773,  2.3613],
        [ 3.4132,  2.4985, -0.8912, -0.2919,  1.3474, -2.3548, -6.5504,  0.1325,
          4.4927,  0.6261],
        [-5.1625, -2.1846, -0.3912, -0.2028,  3.2781,  1.3812, -1.9887, 12.6210,
         -3.2448, -2.0886],
        [-1.3626, -6.5595, -0.5380, -0.4409,  6.3960,  1.1797, -7.4887,  6.8648,
         -0.5992,  1.6634],
        [10.5292,  